# ALTEGRAD Challenge - Classification

*Abderrahim AIT-AZZI, Sébastien OHLEYER, Mickael SUTTON*

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Performing grid search
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime
from log import _check_log_directory,_initialise_model_log

/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data_dir = "./data/"
log_dir = './log'

In [3]:
#initialize model directory
log_name = (datetime.now().strftime('%d-%m-%Y_%H-%M-%S'))
log_filepath = os.path.join(log_dir,log_name,'lighgb.csv')
_check_log_directory(os.path.join(log_dir,log_name))
_initialise_model_log(log_filepath)

Attempting to make log directory at ./log/06-02-2018_12-19-39


### Load data & features

In [4]:
from load_features import load_features

In [5]:
features_train, features_test, data_train = load_features(data_dir)

### Create train and test matrices

In [6]:
X_train= features_train.drop(['is_duplicate'],axis=1)
X_test = features_test
X_train=X_train.replace([np.inf, -np.inf], np.nan)
X_train=X_train.fillna(value=0)
X_test=X_test.replace([np.inf, -np.inf], np.nan)
X_test=X_test.fillna(value=0)
Y_train = data_train["is_duplicate"].values

In [7]:
X_train.columns

Index(['len_q1', 'len_q2', 'diff_len', 'len_char_q1', 'len_char_q2',
       'len_word_q1', 'len_word_q2', 'common_words', 'fuzz_qratio',
       'fuzz_WRatio',
       ...
       'num_v_q1', 'num_v_q2', 'num_s_q1', 'num_s_q2', 'num_u_q1', 'num_u_q2',
       'num_k_q1', 'num_k_q2', 'num_q_q1', 'num_q_q2'],
      dtype='object', length=155)

# Classifier LIGHTGB

In [8]:
from lgb_train import lgb_train

In [9]:
print('Number of features on train matrix: ',len(X_train.columns))
print('Number of features on test matrix: ',len(X_test.columns))

Number of features on train matrix:  155
Number of features on test matrix:  155


** Parameters**

In [10]:
RANDOM_SEED = 2017
lgb_params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'device': 'cpu',
        'feature_fraction': 0.486,
        'num_leaves': 130,
        'lambda_l2': 1.9,
        'learning_rate': 0.01,
        'num_boost_round': 5000,
        'early_stopping_rounds': 50,
        'max_depth': 25,
        'min_data_in_leaf': 15,
        'subsample': 1,
        'colsample_bytree': 1,
        'verbose': 1,
        'bagging_fraction_seed': RANDOM_SEED,
        'feature_fraction_seed': RANDOM_SEED,
    }

In [11]:
feat_imp = lgb_train(X_train, X_test, Y_train, lgb_params, log_filepath, test_prediction=True, num_folds=5)

/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/lightgbm/engine.py:103: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fold 1: 1535 rounds, training loss 0.035988, validation loss 0.116600
Fold 2: 1337 rounds, training loss 0.041199, validation loss 0.122086
Fold 3: 1191 rounds, training loss 0.045550, validation loss 0.123544
Fold 4: 1493 rounds, training loss 0.036772, validation loss 0.121144
Fold 5: 1037 rounds, training loss 0.051163, validation loss 0.127999
Final CV val score: [0.11660031160060474, 0.12208592217243029, 0.12354444149821074, 0.12114379831145441, 0.12799896202824534]
Final mean CV val score: 0.12227468712218911

Make submission file...
Submission file written !


In [14]:
feat_imp[feat_imp['column']=='shortest_path_weighted']

,column,importance
71,shortest_path_weighted,1645


In [ ]:
feat_imp.plot(kind='bar', )

## Important note

The LighGB algorithm has several random initialization and unfortunately, we did not manage to completely fix them to be able to reproduce exactly a prediction. That is why you will not be able to replicate our best submission. We are sincerely apologize for this problem.

However, note that with the provided code you will be able to reproduce every feature, run the classifier and achieve a result really close to our best submission (a random initialization set apart). We are at your disposal to answer any question you might have.